<img style="float: left;" src="./images/PallenceAI-Final.png" width="800">

# Sentiment Analysis on IMDb Reviews

## Sequence Models: BiLSTM

<img style="float: left;" src="./images/sequence-bilstm.png" width="300">

<img style="float: left;" src="./images/imdb2.png" width="400">

### Import needed libraries 

In [1]:
#Basic Python packages for data wrangling
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

from pathlib import Path

#Tensorflow & Keras related packages
import tensorflow as tf
from tensorflow import keras
from keras import layers

from utils import plot_history

### Load IMDb Dataset Preloaded in Keras

In [2]:
import tensorflow_datasets as tfds

C:\Users\sridh\anaconda3\envs\tf2.10_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train, val, test = tfds.load(
    name="imdb_reviews",
    split=["train[:80%]", "train[80%:]", "test"],
    as_supervised=True)

### Understanding the Data 

In [4]:
# Lets see how many sample reviews are there in each dataset

print (len(train))
print (len(val))
print (len(test))

20000
5000
25000


In [5]:
# Batch the data
train_data = train.batch(32)
val_data = val.batch(32)
test_data =test.batch(32)

In [6]:
# lets look at the first batch of data

for reviews, labels in train_data.take(1):
    print ("Reviews shape", reviews.shape)
    print ("Labels shape", labels.shape, "\n")
    
    print ('Sample 7th Review:', reviews[6].numpy().decode("utf-8"), "\n")
    print ('Sample 7th Label:', labels[6].numpy())

Reviews shape (32,)
Labels shape (32,) 

Sample 7th Review: Okay, you have:<br /><br />Penelope Keith as Miss Herringbone-Tweed, B.B.E. (Backbone of England.) She's killed off in the first scene - that's right, folks; this show has no backbone!<br /><br />Peter O'Toole as Ol' Colonel Cricket from The First War and now the emblazered Lord of the Manor.<br /><br />Joanna Lumley as the ensweatered Lady of the Manor, 20 years younger than the colonel and 20 years past her own prime but still glamourous (Brit spelling, not mine) enough to have a toy-boy on the side. It's alright, they have Col. Cricket's full knowledge and consent (they guy even comes 'round for Christmas!) Still, she's considerate of the colonel enough to have said toy-boy her own age (what a gal!)<br /><br />David McCallum as said toy-boy, equally as pointlessly glamourous as his squeeze. Pilcher couldn't come up with any cover for him within the story, so she gave him a hush-hush job at the Circus.<br /><br />and finally

In [7]:
# First select only reviews from training data (which contains both reviews/labels) 
#.....which we will vectorize into numeric format.

train_data_onlyreviews = train_data.map(lambda x,y : x) # given a tuple (x,y), output only x

<img style="float: left;" src="./images/vectorization.png" width="300">

## Data prep for Sequence Models
1. Vectorize the sequences as shown below with TextVectorization in Keras:
* Tokenize the sequences
* Encode the tokens into integers

2. After this we can convert each integer to a corresponding vector (such as one-hot, or word embeddings). This can be 
done by a seperate one-hot layer or embedding layer directly during model building stage

In [8]:
# Keras has inbuilt Text Vectorization layer that can standardize, tokenize, and 
#.....convert to indices or token vectors

In [9]:
# we can also use our own custom functions to pass into Text vectorization

In [10]:
def custom_standardization(input_text):
    # convert to lower case
    lowercase_text = tf.strings.lower(input_text)
    
    #remove html tags
    stripped_html = tf.strings.regex_replace(lowercase_text, "<br\s*/?>", " ")
    
    # Remove most punctuation but keep exclamation marks and question marks
    cleaned_text = tf.strings.regex_replace(stripped_html, "[^a-zA-Z0-9!?]", " ")
    
    return cleaned_text

In [11]:
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization

# this is a limit on sequence length. each sequence will be 300 words long
max_length = 300 

# this is for the vocabulary limit. max tokens to use for creating the vocabulary.
max_tokens = 10000 

text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode = "int",
    output_sequence_length = max_length,
    standardize = custom_standardization
)

In [12]:
# Apply vectorization on training data reviews
# apply text vectorization on training data reviews to index the vocabulary

text_vectorization.adapt(train_data_onlyreviews) 

In [13]:
# vocabulary
vocab = text_vectorization.get_vocabulary()
vocab

['',
 '[UNK]',
 'the',
 'and',
 'a',
 'of',
 'to',
 'is',
 'it',
 'in',
 'i',
 'this',
 'that',
 's',
 'was',
 'as',
 'for',
 'with',
 'movie',
 'but',
 'film',
 't',
 'on',
 'you',
 'not',
 'he',
 'his',
 'are',
 'have',
 'be',
 'one',
 'all',
 'at',
 'they',
 'by',
 'who',
 'an',
 'so',
 'from',
 'like',
 'there',
 'her',
 'or',
 'just',
 'about',
 'out',
 'if',
 'has',
 'what',
 'some',
 'good',
 'can',
 'she',
 'very',
 'when',
 'more',
 'up',
 'would',
 'even',
 'time',
 'my',
 'no',
 'which',
 'story',
 'only',
 'really',
 'their',
 'see',
 'had',
 'we',
 'were',
 'me',
 'well',
 'than',
 'much',
 'been',
 'get',
 'people',
 'will',
 'also',
 'bad',
 'do',
 'other',
 'into',
 'first',
 'because',
 'great',
 'how',
 'don',
 'most',
 'him',
 'made',
 'its',
 'then',
 'make',
 'way',
 'could',
 'them',
 'too',
 'any',
 'after',
 'movies',
 'think',
 'characters',
 'character',
 'two',
 'watch',
 'films',
 'many',
 'being',
 'seen',
 'never',
 'plot',
 'life',
 'love',
 'acting',
 'l

In [14]:
# Vocabulary as a dictionary

vocab_dict = {i:word for i,word in enumerate(vocab)}
vocab_dict

{0: '',
 1: '[UNK]',
 2: 'the',
 3: 'and',
 4: 'a',
 5: 'of',
 6: 'to',
 7: 'is',
 8: 'it',
 9: 'in',
 10: 'i',
 11: 'this',
 12: 'that',
 13: 's',
 14: 'was',
 15: 'as',
 16: 'for',
 17: 'with',
 18: 'movie',
 19: 'but',
 20: 'film',
 21: 't',
 22: 'on',
 23: 'you',
 24: 'not',
 25: 'he',
 26: 'his',
 27: 'are',
 28: 'have',
 29: 'be',
 30: 'one',
 31: 'all',
 32: 'at',
 33: 'they',
 34: 'by',
 35: 'who',
 36: 'an',
 37: 'so',
 38: 'from',
 39: 'like',
 40: 'there',
 41: 'her',
 42: 'or',
 43: 'just',
 44: 'about',
 45: 'out',
 46: 'if',
 47: 'has',
 48: 'what',
 49: 'some',
 50: 'good',
 51: 'can',
 52: 'she',
 53: 'very',
 54: 'when',
 55: 'more',
 56: 'up',
 57: 'would',
 58: 'even',
 59: 'time',
 60: 'my',
 61: 'no',
 62: 'which',
 63: 'story',
 64: 'only',
 65: 'really',
 66: 'their',
 67: 'see',
 68: 'had',
 69: 'we',
 70: 'were',
 71: 'me',
 72: 'well',
 73: 'than',
 74: 'much',
 75: 'been',
 76: 'get',
 77: 'people',
 78: 'will',
 79: 'also',
 80: 'bad',
 81: 'do',
 82: 'other

In [15]:
# '' is used to represent something which is not really a word. 
#...Like for example padding or masking. Its index is 0

# [UNK]' is used to represent an Out of Vocabular token OOV token. Its index is typically 1
# rest of the indices are used for all other words.

In [16]:
# Each review is standardized, tokenized, and convered into an integer sequence

train_int = train_data.map(lambda x,y: (text_vectorization(x),y)) 
val_int = val_data.map(lambda x,y: (text_vectorization(x),y)) 
test_int = test_data.map(lambda x,y: (text_vectorization(x),y)) 

In [17]:
# lets look again at the first batch of data, now that reviews are converted into integers.

for reviews, labels in train_int.take(1):
    print ("Reviews shape", reviews.shape)
    print ("Labels shape", labels.shape, "\n")
        
    print ('Encoded 7th Review:', reviews[6], "\n")    
    print ('Encoded 16th Review:', reviews[15], "\n")

# Each batch has 32 reviews. Each review is converted into a 300 length integer sequence. 

Reviews shape (32, 300)
Labels shape (32,) 

Encoded 7th Review: tf.Tensor(
[ 855   23   28 6875 4650   15  711    1    1  485  485  773    1    5
 1850   52   13  561  123    9    2   84  133   12   13  220 1652   11
  121   47   61    1  764  702 8773   15 7112 3806    1   38    2   84
  324    3  153    2    1 1514    5    2    1 9279    1   15    2    1
  730    5    2    1  883  157 1135   73    2 3806    3  883  157  507
   41  200 2456   19  130    1 8181 8578   24 1937  196    6   28    4
 2940  424   22    2  493    8   13 2796   33   28    1    1   13  361
 1810    3    1   33  229   58  266 2199   16    1  130   52   13    1
    5    2 3806  196    6   28  301 2940  424   41  200  554   48    4
    1  581    1   15  301 2940  424 1284   15    1    1   15   26 9198
    1  421   21  212   56   17   99 1134   16   90  742    2   63   37
   52  508   90    4    1    1  295   32    2 5508    3  411 2507    1
   15  711    1    1    1    1    5    2    1  503 9844  390 4636 3255
 

In [18]:
# zeros in the above review are for padding so that each review can reach max length= 300 words

# 1's in above review represent unknown words. 
#...meaning these words in the review were not there in the vocabulary
#.. since we have chosen max tokens = 10000

In [19]:
# Lets see the encoded review and the review decoded back to text
for reviews, labels in train_int.take(1):     
    print ('Encoded 7th Review:', reviews[6], "\n")    
    
    decoded_review =  " ".join([vocab_dict[int_token] for int_token in reviews[6].numpy()])
    print ('Decoded 7th Review:', decoded_review, "\n")

Encoded 7th Review: tf.Tensor(
[ 855   23   28 6875 4650   15  711    1    1  485  485  773    1    5
 1850   52   13  561  123    9    2   84  133   12   13  220 1652   11
  121   47   61    1  764  702 8773   15 7112 3806    1   38    2   84
  324    3  153    2    1 1514    5    2    1 9279    1   15    2    1
  730    5    2    1  883  157 1135   73    2 3806    3  883  157  507
   41  200 2456   19  130    1 8181 8578   24 1937  196    6   28    4
 2940  424   22    2  493    8   13 2796   33   28    1    1   13  361
 1810    3    1   33  229   58  266 2199   16    1  130   52   13    1
    5    2 3806  196    6   28  301 2940  424   41  200  554   48    4
    1  581    1   15  301 2940  424 1284   15    1    1   15   26 9198
    1  421   21  212   56   17   99 1134   16   90  742    2   63   37
   52  508   90    4    1    1  295   32    2 5508    3  411 2507    1
   15  711    1    1    1    1    5    2    1  503 9844  390 4636 3255
    9   41 1035   17    4 7686    5  945 2910 

## 1. Build the Model with One-hot Vectors

In [20]:
# Reset the seeds for model initialization/training for reproducibility

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

<img style="float: left;" src="./images/onehot2.png" width="700">

In [21]:
hidden_units = 16

def model_BiLSTM(): 
    # Define Input shape
    inputs = keras.Input(shape = (max_length,), dtype="int64") 
    
    # each token in input sequence will be converted to one-hot vector
    onehot = tf.one_hot(inputs, depth=max_tokens) 
    
    # Bidirectional LSTM layer
    x = layers.Bidirectional(layers.LSTM(hidden_units))(onehot)  

    # Dropout Layer
    x = layers.Dropout(0.5)(x) 

    # Dense output Layer
    outputs = layers.Dense(1, activation="sigmoid")(x) 
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [22]:
model = model_BiLSTM()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 300)]             0         
                                                                 
 tf.one_hot (TFOpLambda)     (None, 300, 10000)        0         
                                                                 
 bidirectional (Bidirectiona  (None, 32)               1282176   
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 1,282,209
Trainable params: 1,282,209
Non-trainable params: 0
___________________________________________________

In [23]:
# when we use bidirectional RNN or LSTM, we will have an extra weight matrices 
#...for the backward RNN or LSTM. 
# Extra W for x, a and bias term. 

In [24]:
len(model.weights)

8

In [25]:
model.weights[0].shape # weight matrix that will be multiplied with Xt

TensorShape([10000, 64])

In [26]:
model.weights[1].shape # weight matrix that will be multiplied with at-1 (activation)

TensorShape([16, 64])

In [27]:
model.weights[2].shape # bias term ba

TensorShape([64])

In [28]:
model.weights[3].shape # weight matrix that will be multiplied with Xt - for backward lstm

TensorShape([10000, 64])

In [29]:
model.weights[4].shape # weight matrix that will be multiplied with at-1 (activation) - for backward lstm

TensorShape([16, 64])

In [30]:
model.weights[5].shape # bias term ba - for backward lstm

TensorShape([64])

In [31]:
model.weights[6].shape # weight matrix that will be multiplied with at to calculate dense layer output (yt)

TensorShape([32, 1])

In [32]:
model.weights[7].shape # bia term by-  to calculate dense layer output (yt)

TensorShape([1])

### Compile & Train the Model

In [33]:
model = model_BiLSTM()
path = Path("./models/model_bilstm.keras")

In [34]:
# We will reuse this function to train and evaluate for convenience

def train_evaluate(model,path,train,val,test):
    
    # Call backs
    checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath = path,
                                                       save_best_only=True) # Save only best model
    
    earlystop_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    callbacks = [checkpoint_cb,earlystop_cb]

    # Compile the model
    model.compile(optimizer="rmsprop", loss='binary_crossentropy',  metrics = ["accuracy"])
    
    #Train the model
    history = model.fit(train, validation_data = val, callbacks=callbacks, epochs=20)
    
    #Evaluate the model on test data
    test_loss, test_accuracy = model.evaluate(test)
    
    return (history,test_accuracy)

In [ ]:
(history_bilstm,test_accuracy_bilstm) = train_evaluate(model,path,
                                                       train_int,
                                                       val_int,
                                                       test_int)

**Plot History**

In [ ]:
plot_history(history_bilstm,"accuracy")

In [ ]:
# Mean absolute error with this simple 1 neuron dense model 
print (f"Accuracy on the test data set is {test_accuracy_bilstm}")

## 2. Build the Model with Word Embeddings

In [ ]:
# One-hot vectors are sparse and very high dimensional - Vocab length
# word embeddings can help with these in that they are more meaningul, and 
#....are typically smaller dimensions like 128, 256, 512

# words with similar meaning have embedding vectors are that are closer and similar

In [ ]:
# Reset the seeds for model initialization/training for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

<img style="float: left;" src="./images/wordembed3.png" width="800">

In [ ]:
hidden_units = 16

def model_BiLSTM_wembeddings(): 
    
    # Define Input shape
    inputs = keras.Input(shape = (max_length,), dtype="int64") 
    
    # Each token in input sequence will be converted to a 128 dimensional word embedding vector.
    embeddings = layers.Embedding(input_dim = max_tokens, output_dim=128, mask_zero=True)(inputs) 
     
    # Bidirectional LSTM layer 
    x = layers.Bidirectional(layers.LSTM(hidden_units))(embeddings) 

    # Dropout Layer
    x = layers.Dropout(0.5)(x) 

    # Dense output Layer
    outputs = layers.Dense(1, activation="sigmoid")(x) 
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# what makes a good word embedding depends heavily on the particular task.
# If we use an embedding layer like above, word embeddings are calculated automatically 
#...as part of learning process
# mask_Zero = True, above ensures extra zeros added for padding shorter sequences are ignored 

In [ ]:
model = model_BiLSTM_wembeddings()
model.summary()

In [ ]:
len(model.weights)

In [ ]:
model.weights[0].shape # Word Embedding matrix weights that will be calculated as part of training

In [ ]:
model.weights[1].shape  # weight matrix that will be multiplied with Xt

In [ ]:
model.weights[2].shape # weight matrix that will be multiplied with at-1 (activation)

In [ ]:
model.weights[3].shape # bias term ba

In [ ]:
model.weights[4].shape # weight matrix that will be multiplied with Xt -  backward lstm

In [ ]:
model.weights[5].shape # weight matrix that will be multiplied with at-1 (activation) - backward lstm

In [ ]:
model.weights[6].shape # bias term ba -  backward lstm

In [ ]:
model.weights[7].shape # # weight matrix that will be multiplied with at to calculate dense layer output (yt)

In [ ]:
model.weights[8].shape # bias term by -  to calculate dense layer output (yt)

### Compile & Train the Model

In [ ]:
model = model_BiLSTM_wembeddings()
path = Path("./models/model_bilstm_wembeddings.keras")

In [ ]:
#We will reuse this function to train and evaluate for convenience
def train_evaluate(model,path,train,val,test):
    
    #call backs
    checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath = path,
                                                       save_best_only=True) # Save only best model
    
    earlystop_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    callbacks = [checkpoint_cb,earlystop_cb]

    #Compile the model
    model.compile(optimizer="rmsprop", loss='binary_crossentropy',  metrics = ["accuracy"])
    
    #Train the model
    history = model.fit(train, validation_data = val, callbacks=callbacks, epochs=10)
    test_loss, test_accuracy = model.evaluate(test)
    
    return (history,test_accuracy)

In [ ]:
(history_bilstm_wembeddings,test_accuracy_bilstm_wembeddings) = train_evaluate(model,path,train_int,val_int,test_int)

In [ ]:
print (f"Accuracy on the test data set is {test_accuracy_bilstm_wembeddings}")

## 3. Pretrained Word Embeddings (Optional)

In [ ]:
# When we dont have enough training data to  learn task specific word embeddings
# when we think pretrained word embeddings on some other task are fairly generic to apply to our task as well
# similar to pretrained features in convnets

# Many different pretrained word embeddings. some generic and some domain specific
* Word2vec
* Glove

### Download Pretrained Glove Embeddings

In [ ]:
# http://nlp.stanford.edu/data/glove.6B.zip
#unzip

In [ ]:
embeddings_index = {}
file_path = Path("./data/glove.6B/glove.6B.50d.txt")
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
# Format the loaded word embeddings into an embedding index which will be a dictionary 
# whose keys will be words and values will be 50-dimensional wordembeddings

In [ ]:
len(embeddings_index) # Pretrained word embeddings for around 400000 words

### Create Pretrained Glove Word Embedding Matrix

In [ ]:
# create the pretrain word embedding matrix from the embeddings_index. Dimensions will be of size (vocab_size,embedding_dim)
input_dim = max_tokens
output_dim = 50
embedding_matrix = np.zeros((input_dim,output_dim))

for i, word in enumerate(vocab):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

### Build the Model

In [ ]:
hidden_units = 16

def model_BiLSTM_wembeddings_pretrained(): 
    
    # Define Input shape
    inputs = keras.Input(shape = (max_length,), dtype="int64") 
    
    # Each token in input sequence will be converted to a 50-dim pretrained word embedding vector. 
    embeddings =layers.Embedding(input_dim,output_dim,weights=[embedding_matrix],
                                 trainable=False, mask_zero=True)(inputs) 
    
    # Bidirectional LSTM layer 
        x = layers.Bidirectional(layers.LSTM(hidden_units))(embeddings) 

    # Dropout Layer
    x = layers.Dropout(0.5)(x) 

    # Dense output Layer
    outputs = layers.Dense(1, activation="sigmoid")(x) 
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
model = model_BiLSTM_wembeddings_pretrained()
model.summary()

In [ ]:
#We will reuse this function to train and evaluate for convenience
def train_evaluate(model,path,train,val,test):
    
    #call backs
    checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath = path,
                                                       save_best_only=True) # Save only best model
    
    earlystop_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, 
                                                 restore_best_weights=True)
    callbacks = [checkpoint_cb,earlystop_cb]

    #Compile the model
    model.compile(optimizer="rmsprop", loss='binary_crossentropy',  metrics = ["accuracy"])
    
    #Train the model
    history = model.fit(train, validation_data = val, callbacks=callbacks, epochs=10)
    test_loss, test_accuracy = model.evaluate(test)
    
    return (history,test_accuracy)

In [ ]:
(history_bilstm_wembeddings_pt,test_accuracy_bilstm_wembeddings_pt) = train_evaluate(model,path,
                                                                                     train_int,
                                                                                     val_int,test_int)

In [ ]:
print (f"Accuracy on the test data set is {test_accuracy_bilstm_wembeddings_pt}")